In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.1'
spark_version = 'spark-3.4.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,342 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,266 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 2,950 kB in 4s (703 kB/s)
Reading package list

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

In [4]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')

In [6]:
spark.sql("""
select *
from home_sales
limit 5
""").show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [10]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""
SELECT YEAR(date) AS year, ROUND(AVG(price),2) AS average_price_4_Bedroom_Homes
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR(date)
ORDER BY year
""").show()


+----+-----------------------------+
|year|average_price_4_Bedroom_Homes|
+----+-----------------------------+
|2019|                     300263.7|
|2020|                    298353.78|
|2021|                    301819.44|
|2022|                    296363.88|
+----+-----------------------------+



In [11]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""
SELECT YEAR(date) AS year, ROUND(AVG(price),2) AS average_price_3_bed_3_bath
FROM home_sales
WHERE bedrooms = 3 AND
bathrooms = 3
GROUP BY YEAR(date)
ORDER BY year
""").show()


+----+--------------------------+
|year|average_price_3_bed_3_bath|
+----+--------------------------+
|2019|                 287287.82|
|2020|                 294204.16|
|2021|                 294211.46|
|2022|                 292725.69|
+----+--------------------------+



In [25]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""
SELECT YEAR(date) AS year, ROUND(AVG(price),2) AS avg_price_3_bed_3_bath_2_floors_2000sqft
FROM home_sales
WHERE bedrooms = 3 AND
bathrooms = 3 AND
floors = 2 AND
sqft_living >= 2000
GROUP BY YEAR(date)
ORDER BY year
""").show()


+----+----------------------------------------+
|year|avg_price_3_bed_3_bath_2_floors_2000sqft|
+----+----------------------------------------+
|2019|                               289859.14|
|2020|                               292289.09|
|2021|                               296330.96|
|2022|                               290242.99|
+----+----------------------------------------+



In [14]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
SELECT ROUND(AVG(view), 2) AS average_view_rating
FROM home_sales
WHERE price >= 350000
""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+-------------------+
|average_view_rating|
+-------------------+
|              32.26|
+-------------------+

--- 1.6204349994659424 seconds ---


In [15]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [16]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [17]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""
SELECT ROUND(AVG(view), 2) AS average_view_rating
FROM home_sales
WHERE price >= 350000
""").show()

print("--- %s seconds ---" % (time.time() - start_time))


+-------------------+
|average_view_rating|
+-------------------+
|              32.26|
+-------------------+

--- 0.4901094436645508 seconds ---


In [18]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.parquet('parquet_on_date_built', mode='overwrite')

In [19]:
# 11. Read the parquet formatted data.
p_df=spark.read.parquet('parquet_on_date_built')

In [20]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('home_sales_parquet')

In [21]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""
SELECT ROUND(AVG(view), 2) AS average_view_rating
FROM home_sales
WHERE price >= 350000
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------------+
|average_view_rating|
+-------------------+
|              32.26|
+-------------------+

--- 0.2446298599243164 seconds ---


In [22]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [23]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False